# Why/Motivation: 

1. I was really intrigued by the recent development of Anthropic's chatbot, Claude, a supposedly more ethical chatbot compared to openAI's, chatGPT. I really wanted to test its capabilities of being more "ethical" to see if it was measuring up to its value proposition.

2. I wanted to do sentiment analysis of responses across the world wide web to see which one is most trusted.

3. Good for me to get updated on NLP techniques.

4. Fun to see how the world thinks of chatGPT/Claude and what they think of themselves. (turning Q & A on themselves. self-introspection)

5. Wanted to see if can do sentiment analysis across multiple emotions to not have such a Black and White assessment of statements since feelings/people's positions are usually not polar, but spead across a variety of feelings.

6. Not just explore Twitter (X) and Reddit, but also pop songs for real-time relevant responses.

7. Look at fresh time series data.

8. Explore a different viz py lib: d3.js to build more sophisticated, interactive viz for web portfolio

# Business Recommendation:

Thought this would be a nice twist on sentiment analysis on product launches. This is something I was looking to do since coming from product analytics, experimentation looked at event signals versus what people were actually saying about the product. People say that actions are louder than words, but words also speak volumes. Would have been nice to have complemented by experimentation resuslts with some sort of sentiment analysis. Would have added more weight, been more comprehensive assessment of new feature launches.

# 1. Define metrics of success (its value prop, for business) for helpful chatbot (use cases)

# 2. Scrape:
* ~Twitter~ have to pay for scrape; no bueno
* Reddit (1000 cap)
* Threads
* BlueSky
* Mastodon (spam, search sucks)?
* Substack/Substack Notes
* Medium
* YouTube
* LinkedIn
* Song Lyrics

to get info on following chatbots to do analysis on:
* chatGPT (OpenAI)
* Claude (Anthropic)
* Bard (Google)
* Bing Chat (Microsoft, images, voice chat)
* Perplexity AI (advancing how people discover and share information)
* Pi (empathetic/human emotions)

# chatGPT Comments

# 3. Complete d3.js visualizations on emotions

# 4. Other NLP-related analyses (Topic Modeling)

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json

In [2]:
# credentials = {}
# try:
#     with open('/kaggle/input/credentials/credentials.json') as file:
#         credentials = json.load(file)    
# except FileNotFoundError:
#     print("Error: credentials.json file not found.")
    
# print(credentials["youtube_api_key"])

In [3]:
credentials = json.load(open('/kaggle/input/credentials/credentials.json'))

# Next iteration, get >100 comments

In [4]:
# Ref: https://github.com/analyticswithadam/Python/blob/main/YouTubeComments.ipynb

import googleapiclient.discovery
import googleapiclient.errors

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = credentials["youtube_api_key"]

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part="snippet",
    videoId="HrCIWSUXRmo",
    maxResults=1000
)
response = request.execute()

comments = []

for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    comments.append([
        comment['authorDisplayName'],
        comment['publishedAt'],
        comment['updatedAt'],
        comment['likeCount'],
        comment['textDisplay']
    ])

df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])

# df.head(10)

In [5]:
# Peak
df.head(10)

,author,published_at,updated_at,like_count,text
0,Tangent Fox,2023-10-31T04:51:26Z,2023-10-31T04:51:43Z,1,"<a href=""https://www.youtube.com/watch?v=HrCIWSUXRmo&amp;t=9m54s"">9:54</a> Haha, I&#39;m a GPT at standardized tests.."
1,Tangent Fox,2023-10-31T04:30:24Z,2023-10-31T04:30:24Z,1,"I am curious... is anyone actually surprised that LLMs are bad at logical deduction? LLMs are really really exceptionally good autocomplete, so of course they struggle with logic.<br><br>It&#39;s like saying a textbook that explains logical deduction can perform logical deduction. No, it&#39;s text. Text doesn&#39;t do anything on its own."
2,Leslie Viljoen,2023-10-24T00:31:38Z,2023-10-24T00:31:38Z,1,I just tried &quot;a horse drinking from a water bottle&quot; and Dall-e 3 did very well. Lucky me!
3,dosmastrify,2023-10-12T13:18:15Z,2023-10-12T13:18:15Z,0,Is the singularity near?
4,Alex Mort,2023-10-09T10:47:49Z,2023-10-09T10:47:49Z,1,Chat GPT: Eg e fra Bergen
5,Chad Valentine,2023-10-08T17:30:02Z,2023-10-08T17:30:02Z,0,Your question is terrible. Its poorly formed. No wonder it couldnt answer. &quot;Do not deny that you know nothing&quot; Dont you mean &quot;Dont deny that you know of&quot;
6,Lystic,2023-10-07T05:56:18Z,2023-10-07T06:05:08Z,0,"A = B and B = A aren&#39;t necessarily the same relationship.<br>I teach an AI that 2+2 = 4. Then I show it 4 donuts. One donut spare and 3 in a bag. That&#39;s not 2+2 donuts, it&#39;s 1+3 donuts. For humans this is not easy to intuit as it&#39;s naturally understood.There are many combinations that make 4 donuts. And 4 donuts can mean many combinations. It&#39;s not the same relationship in practice. So maybe that&#39;s why it doesn&#39;t simply equate the two both ways. If 2+2 means 4, it shouldn&#39;t assume that 4 means 2+2.<br><br>A bicycle is a form of transportation. It doesn&#39;t equal a form of transportation. (A bicycle is many more things than a mode of transportation.) But it is. And a form of transportation doesn&#39;t equal and isn&#39;t necessarily a bicycle. The world is full of this."
7,DankMemer420,2023-10-06T23:36:11Z,2023-10-06T23:36:11Z,0,You just click baited my ass 😂
8,Con K,2023-10-06T22:54:44Z,2023-10-06T22:54:44Z,1,I wonder if we cant teach it to code basic logic for itself with code interpreter? Its pretty insane what it can help program.
9,wishsnfishs,2023-10-06T18:03:01Z,2023-10-06T18:03:01Z,1,I dont understand why LLMs cant review their own responses before submitting them - considering prompting them to do so usually improves the quality. Humans ourselves already overhear and evaluate our own speech as we produce it - imagine how garbled our communication would be if we weren&#39;t aware of what we&#39;d said until we stopped talking


# Add sentiment
## Candidates:
* Vader
* Happy Transformer
* TextBlob
* Google NL API

In [6]:
# Vader
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

df_vader = df.copy()
analyzer = SentimentIntensityAnalyzer()
df_vader['rating'] = df_vader['text'].apply(analyzer.polarity_scores)

# Docs on how scoring was derived and what it means: https://github.com/cjhutto/vaderSentiment#about-the-scoring
df_vader['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df_vader['text']]
df_vader['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df_vader['text']]
df_vader['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df_vader['text']]
df_vader['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df_vader['text']]

/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
df_vader.head()

,author,published_at,updated_at,like_count,text,rating,compound,neg,neu,pos
0,Tangent Fox,2023-10-31T04:51:26Z,2023-10-31T04:51:43Z,1,"<a href=""https://www.youtube.com/watch?v=HrCIWSUXRmo&amp;t=9m54s"">9:54</a> Haha, I&#39;m a GPT at standardized tests..","{'neg': 0.0, 'neu': 0.7, 'pos': 0.3, 'compound': 0.4588}",0.4588,0.000,0.700,0.300
1,Tangent Fox,2023-10-31T04:30:24Z,2023-10-31T04:30:24Z,1,"I am curious... is anyone actually surprised that LLMs are bad at logical deduction? LLMs are really really exceptionally good autocomplete, so of course they struggle with logic.<br><br>It&#39;s like saying a textbook that explains logical deduction can perform logical deduction. No, it&#39;s text. Text doesn&#39;t do anything on its own.","{'neg': 0.138, 'neu': 0.722, 'pos': 0.14, 'compound': 0.0348}",0.0348,0.138,0.722,0.140
2,Leslie Viljoen,2023-10-24T00:31:38Z,2023-10-24T00:31:38Z,1,I just tried &quot;a horse drinking from a water bottle&quot; and Dall-e 3 did very well. Lucky me!,"{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'compound': 0.6969}",0.6969,0.000,0.693,0.307
3,dosmastrify,2023-10-12T13:18:15Z,2023-10-12T13:18:15Z,0,Is the singularity near?,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,0.000,1.000,0.000
4,Alex Mort,2023-10-09T10:47:49Z,2023-10-09T10:47:49Z,1,Chat GPT: Eg e fra Bergen,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,0.000,1.000,0.000


In [8]:
# Happy Transformer
df_ht = df.copy()

In [9]:
!pip install -q transformers
from transformers import pipeline

# Set up the inference pipeline using a model from the 🤗 Hub
sentiment_analysis = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

In [10]:
test = df_ht.head(5)

In [11]:
test['result'] = test['text'].apply(lambda x: sentiment_analysis(x))
test.head()

/tmp/ipykernel_20/172617282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['result'] = test['text'].apply(lambda x: sentiment_analysis(x))


,author,published_at,updated_at,like_count,text,result
0,Tangent Fox,2023-10-31T04:51:26Z,2023-10-31T04:51:43Z,1,"<a href=""https://www.youtube.com/watch?v=HrCIWSUXRmo&amp;t=9m54s"">9:54</a> Haha, I&#39;m a GPT at standardized tests..","[{'label': 'NEU', 'score': 0.9204737544059753}]"
1,Tangent Fox,2023-10-31T04:30:24Z,2023-10-31T04:30:24Z,1,"I am curious... is anyone actually surprised that LLMs are bad at logical deduction? LLMs are really really exceptionally good autocomplete, so of course they struggle with logic.<br><br>It&#39;s like saying a textbook that explains logical deduction can perform logical deduction. No, it&#39;s text. Text doesn&#39;t do anything on its own.","[{'label': 'NEG', 'score': 0.9524347186088562}]"
2,Leslie Viljoen,2023-10-24T00:31:38Z,2023-10-24T00:31:38Z,1,I just tried &quot;a horse drinking from a water bottle&quot; and Dall-e 3 did very well. Lucky me!,"[{'label': 'POS', 'score': 0.979310154914856}]"
3,dosmastrify,2023-10-12T13:18:15Z,2023-10-12T13:18:15Z,0,Is the singularity near?,"[{'label': 'NEU', 'score': 0.866348147392273}]"
4,Alex Mort,2023-10-09T10:47:49Z,2023-10-09T10:47:49Z,1,Chat GPT: Eg e fra Bergen,"[{'label': 'NEU', 'score': 0.963374137878418}]"


In [12]:
test = (
test.assign(sentiment = lambda x: x['text'].apply(lambda s: sentiment_analysis(s)))
    .assign(
         label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

test.head()

,author,published_at,updated_at,like_count,text,result,sentiment,label,score
0,Tangent Fox,2023-10-31T04:51:26Z,2023-10-31T04:51:43Z,1,"<a href=""https://www.youtube.com/watch?v=HrCIWSUXRmo&amp;t=9m54s"">9:54</a> Haha, I&#39;m a GPT at standardized tests..","[{'label': 'NEU', 'score': 0.9204737544059753}]","[{'label': 'NEU', 'score': 0.9204737544059753}]",NEU,0.920474
1,Tangent Fox,2023-10-31T04:30:24Z,2023-10-31T04:30:24Z,1,"I am curious... is anyone actually surprised that LLMs are bad at logical deduction? LLMs are really really exceptionally good autocomplete, so of course they struggle with logic.<br><br>It&#39;s like saying a textbook that explains logical deduction can perform logical deduction. No, it&#39;s text. Text doesn&#39;t do anything on its own.","[{'label': 'NEG', 'score': 0.9524347186088562}]","[{'label': 'NEG', 'score': 0.9524347186088562}]",NEG,0.952435
2,Leslie Viljoen,2023-10-24T00:31:38Z,2023-10-24T00:31:38Z,1,I just tried &quot;a horse drinking from a water bottle&quot; and Dall-e 3 did very well. Lucky me!,"[{'label': 'POS', 'score': 0.979310154914856}]","[{'label': 'POS', 'score': 0.979310154914856}]",POS,0.979310
3,dosmastrify,2023-10-12T13:18:15Z,2023-10-12T13:18:15Z,0,Is the singularity near?,"[{'label': 'NEU', 'score': 0.866348147392273}]","[{'label': 'NEU', 'score': 0.866348147392273}]",NEU,0.866348
4,Alex Mort,2023-10-09T10:47:49Z,2023-10-09T10:47:49Z,1,Chat GPT: Eg e fra Bergen,"[{'label': 'NEU', 'score': 0.963374137878418}]","[{'label': 'NEU', 'score': 0.963374137878418}]",NEU,0.963374


Most likely will not use Hugging Face's Sentiment Analysis model since it has a vocab limit. So not even 100 comments from YouTube vid can be classified

In [13]:
# TextBlob 
from textblob import TextBlob

df_tb = df.copy()
#The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). 
#The polarity score is a float within the range [-1.0, 1.0]. 
#The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
df_tb['sentiment'] = df_tb['text'].apply(lambda x: TextBlob(x).sentiment)
df_tb['polarity'] = df_tb['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df_tb['subjectivity'] = df_tb['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_tb.head()

,author,published_at,updated_at,like_count,text,sentiment,polarity,subjectivity
0,Tangent Fox,2023-10-31T04:51:26Z,2023-10-31T04:51:43Z,1,"<a href=""https://www.youtube.com/watch?v=HrCIWSUXRmo&amp;t=9m54s"">9:54</a> Haha, I&#39;m a GPT at standardized tests..","(0.2, 0.3)",0.200000,0.300000
1,Tangent Fox,2023-10-31T04:30:24Z,2023-10-31T04:30:24Z,1,"I am curious... is anyone actually surprised that LLMs are bad at logical deduction? LLMs are really really exceptionally good autocomplete, so of course they struggle with logic.<br><br>It&#39;s like saying a textbook that explains logical deduction can perform logical deduction. No, it&#39;s text. Text doesn&#39;t do anything on its own.","(0.16875, 0.6145833333333333)",0.168750,0.614583
2,Leslie Viljoen,2023-10-24T00:31:38Z,2023-10-24T00:31:38Z,1,I just tried &quot;a horse drinking from a water bottle&quot; and Dall-e 3 did very well. Lucky me!,"(0.30833333333333335, 0.5666666666666667)",0.308333,0.566667
3,dosmastrify,2023-10-12T13:18:15Z,2023-10-12T13:18:15Z,0,Is the singularity near?,"(0.1, 0.4)",0.100000,0.400000
4,Alex Mort,2023-10-09T10:47:49Z,2023-10-09T10:47:49Z,1,Chat GPT: Eg e fra Bergen,"(0.0, 0.0)",0.000000,0.000000


In [14]:
# Google NL API

# df_gnl = df.copy()
# # Imports the Google Cloud client library
# from google.cloud import language_v1

# # Instantiates a client
# client = language_v1.LanguageServiceClient()

# # The text to analyze
# text = df_gnl['text']
# document = language_v1.Document(
#     content=text, type_ = language_v1.Document.Type.PLAIN_TEXT
# )

# # Detects the sentiment of the text
# sentiment = client.analyze_sentiment(
#     request={"document": document}
# ).document_sentiment

In [15]:
# Add categories for analytics
# df['chatbot'] = 'chatGPT'
# df['scraped'] = 'YouTube'